In [1]:
import tqdm
import snscrape.modules.twitter as tweet
import pandas as pd
import datetime
import re

In [ ]:


# Get Tweets

query1 = "Northeastern events"
query2 = "Northeastern oncampusemployment"
query3 = "Northeastern Co-op Opportunities"
query4 = "Northeastern University"

querylist = [query1, query2, query3, query4]
li = []

for i in querylist:
    count = 0
    for t in tweet.TwitterHashtagScraper(i).get_items():
        tweet_map={}
        a = vars(t)
        #print (a.keys())
        #print (a['content']+ " ====>"+ a['url'])
        tweet_map['TwitterID']=str(a['id'])
        tweet_map['TwitterHandle']=a['user']
        tweet_map['TwitterText']=a['content']
        tweet_map['HashTags']=a['hashtags']
        tweet_map['CreatedAt']=a['date']
        tweet_map['likes'] = a['likeCount']
        tweet_map["replies"] = a['replyCount']
        tweet_map['retweet'] = a["retweetCount"]
        tweet_map['Query']= i
        li.append(tweet_map)

        if count==50:
            break
        count+=1
    
df = pd.DataFrame(li)
df.to_csv("D:/Darshana//NEU/Study/DMDD/Project/MStudents/Tweets.csv",index=False)



In [2]:
import mysql.connector
conn = mysql.connector.connect(host='localhost',
                        database='mastersstudents',
                        user='root',
                        password='root')

cursor = conn.cursor()

#Executing an MYSQL function using the execute() method
cursor.execute("SELECT DATABASE()")

# Fetch a single row using fetchone() method.
data = cursor.fetchone()
print("Connection established to: ",data)
cursor = conn.cursor()

Connection established to:  ('mastersstudents',)


In [ ]:



df = pd.read_csv("D:/Darshana//NEU/Study/DMDD/Project/MStudents/Tweets.csv")

def get_user_details(username):
    user_detail = {}
    user = tweet.TwitterUserScraper(username)
    user = user.entity
    user_detail["full_name"] = user.displayname
    user_detail["username"] = user.username
    user_detail["profile_image"] = user.profileImageUrl
    user_detail["verified"] = user.verified
    user_detail["id"] = user.id
    user_detail["description"] = user.description
    user_detail["createdAt"] = user.created
    user_detail["followers"] = user.followersCount
    user_detail["following"] = user.friendsCount
    return user_detail

c = 0
for i, row in tqdm.tqdm(df.iterrows()):
    try:
        user = row['TwitterHandle'].replace("https://twitter.com/", "")
        user_detail = get_user_details(user)

        query = """insert into twitter(TwitterID, TwitterHandle, UserId, TwitterText,HashTags,CreatedAt,Likes, retweetCount,replyCount,Query) values ({0}, "{1}", {2}, "{3}", "{4}", "{5}" ,{6}, {7}, {8}, "{9}")""".format(int(row['TwitterID']), row['TwitterHandle'], user_detail['id'],row['TwitterText'].replace('\t', '.'), row['HashTags'], row['CreatedAt'], row["likes"], row["retweet"], row["replies"],row['Query'])
        cursor.execute(query)
        
        hashtags = eval(row['HashTags'])
        
        for h in hashtags:
            cursor.execute("""insert into tweet_tags(TwitterID, HashTags) values({0}, "{1}")""".format(row['TwitterID'], h))
        
        query = """insert into TwitterUser(userid, UserName, handle, Name,CreatedAt, Followers_Count,FRIENDS_Count, Account_Verified, Description, Profile_image_url) values ({0}, "{1}", "{2}", "{3}", "{4}", {5}, {6}, "{7}", "{8}", "{9}")""".format(int(user_detail['id']), user_detail['username'], row["TwitterHandle"],user_detail['full_name'], user_detail['createdAt'], int(user_detail['followers']), int(user_detail['following']), user_detail["verified"], user_detail["description"], user_detail["profile_image"])
        cursor.execute(query)
    except mysql.connector.errors.Error as e:
        pass
#         print (e)
    except Exception:
#         print (e)
        c+=1

print (c)
conn.commit()


In [5]:
# sheet_names = ["Fall","Spring","StudentJob", "StudentOnCampus","OnCampusEmployment","OnCampusEmploymentStats"]
sheet_names = ["OnCampusEmploymentStats"]

for sheet in sheet_names:
    df = pd.read_excel("D:\Darshana\\NEU\Study\DMDD\Project\MStudents\Assignment3\Mstudents.xlsx", sheet_name= sheet)
    c = 0
    for i, row in df.iterrows():
        try:
            if sheet == "Student":
                query = """INSERT INTO Student (StudentID, FirstName, LastName, Gender, DeptId, FeePayment, CreditsEarned) VALUES({0}, "{1}", "{2}", "{3}", {4}, "{5}", {6});""".format(int(row['StudentID']), row['FirstName'], row['LastName'], row['Gender'], int(row['DeptID']), row['FeePayment'],int(row['CreditsEarned']))
            
            if sheet == "Fall":
                query = """INSERT INTO Fall (StudentID, IntakeYear,ID) VALUES({0}, {1}, {2})""".format(int(row['StudentID']),int(row['IntakeYear']),int(row['ID']))
                
            if sheet == "Spring":
                
                query = """INSERT INTO Spring (StudentID, IntakeYear,ID) VALUES({0}, {1}, {2})""".format(int(row['StudentID']),int(row['IntakeYear']),int(row['ID']))
                
                
            if sheet == "College":
                query = """INSERT INTO College (CollegeName, DeptName, DeptID, DeanName) VALUES("{0}", "{1}", {2}, "{3}")""".format(row['CollegeName'], row['DeptName'], int(row['DeptID']),row['DeanName'])
                
            if sheet == "EventDetails":
                query = """INSERT INTO EventDetails(EID, EName, EventHeadID, Location, Purpose, Dateandtime, FareinUSD) VALUES({0},"{1}",{2},"{3}","{4}","{5}","{6}")""".format(int(row['EID']), row['EName'], int(row['EventHeadID']), row['Location'],row['Purpose'],datetime.datetime.strptime(row['Dateandtime'].replace(" EST", ""), "%a, %b %d, %Y %I:%M %p"), row['FareinUSD'])
            
            if sheet == "OnCampusEmployment":
                query = """INSERT INTO OnCampusEmployment(ID, JobID, Employer, Location) VALUES({0},{1},"{2}","{3}")""".format(int(['ID']), int(row['JobID']), row['Employer'], row['Location'])
                
            if sheet == "OnCampusEmploymentStats":
                query = """INSERT INTO OnCampusEmploymentStats(ID, JobID, SkillsRequired, SalaryInUSD, DurationInMonths, PostingDate, Deadline, Vacancy) VALUES({0},{1},"{2}",{3},{4},"{5}","{6}",{7})""".format(int(row['ID']), int(row['JobID']), row['SkillsRequired'], int(row['SalaryInUSD']),int(row['DurationInMonths']), datetime.datetime.strptime(row['PostingDate'].replace(" EST", ""), "%a, %b %d, %Y %I:%M %p"), datetime.datetime.strptime(row['Deadline'].replace(" EST", ""), "%a, %b %d, %Y %I:%M %p"),int(row['Vacancy']))
                
            if sheet == "StudentOnCampus":
                query = """INSERT INTO StudentOnCampus(StudentID, JobID,ID) VALUES({0},{1},{2})""".format(int(row['StudentID']), int(row['JobID']),int(row['ID']))

            if sheet == "InternshipsAndCoop":
                salary = row["SalaryEstimate"].replace("(Glassdoor est.)", "").replace("K", "000").replace("$","")
                salary = salary.split("-")
#                 import ipdb; ipdb.set_trace()
                lower_salary = salary[0]
                upper_salary = salary[1]
                rating = row['Rating']
                company_name = row['CompanyName'].replace('\n','').replace(str(rating),'')
                job_description = row['JobDescription'][:900]
                job_description = re.sub(r"\"","\'", job_description)
                job_title = re.sub(r"\"","\'", row['JobTitle'])
                query = """INSERT INTO InternshipsAndCoop(JobID, JobTitle, SalaryEstimateLower, SalaryEstimateUpper, JobDescription, Rating, CompanyName, Location, HeadQuaters, Size, FoundedYear, TypeOfOwnership, Industry, Sector, Revenue, Competitor) VALUES({0}, "{1}",{2},{3},"{4}",{5},"{6}","{7}","{8}","{9}",{10},"{11}","{12}","{13}","{14}","{15}")""".format(int(row['JobID']), job_title, int(lower_salary), int(upper_salary), job_description,float(row['Rating']), company_name,row['Location'], row['HeadQuarters'],row['Size'],int(row['Founded']),row['TypeOfOwnership'],row['Industry'],row['Sector'], row['Revenue'],row['Competitors']) 
                
            if sheet == "StudentClub":
                query = """INSERT INTO StudentClub (ClubID, ClubName, StudentID, Contacts, Motive) VALUES({0},"{1}",{2},"{3}","{4}")""".format(int(row['ClubID']), row['ClubName'], row['StudentID'], row['Contacts'], row['Motive'])
                
            if sheet == "StudentJob":
                query = "INSERT INTO StudentJob (StudentID, JobID, ID) VALUES({0}, {1},{2})".format(int(row['StudentID']), int(row['JobID']),int(row['ID']))
            cursor.execute(query)

        except Exception as e:
            print (e, sheet)
#             print (query)
#             break
            c+=1
    print (c)
    conn.commit()

0


In [ ]:
    #         query = """insert into twitter(TwitterID, TwitterHandle, TwitterText,HashTags,CreatedAt,Query) values ({0}, "{1}", "{2}", "{3}", "{4}", "{5}")""".format(int(row['TwitterID']), row['TwitterHandle'], row['TwitterText'].replace('\t', '.'), row['HashTags'], row['CreatedAt'], row['CreatedAt'])
        #         print (int(row['JobId']), row['Employer'], row['Location'], row['SkillsRequired'],int(row['Salary']),int(row['Duration'].replace(" Months", "")), row['PostingDate'], row['Deadline'], int(row['Vacancy']))